In [4]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# === CONFIG ===
ENTRY_TIME = "09:30:59"
EXIT_TIME = "15:15:00"
SLIPPAGE = 0.005  # per leg
LOTSIZE = 75
BROKERAGE_PER_LEG = 0.5  # per leg per lot
SL_PCT = 0.10
INITIAL_CAPITAL = 1000000
MAX_REENTRIES = 3  # not implemented yet
DATA_FOLDER = r"C:\Users\Shubham lodha\OneDrive\Desktop\Assessment\Options data\APR_2021"

def load_data(folder_path):
    """
    Load and combine all CSV files from the specified folder
    """
    all_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path)
                # Combine Date + Time into timestamp
                df['timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
                # Standardize column names
                df.columns = df.columns.str.lower().str.replace(" ", "_")
                all_data.append(df)
            except Exception as e:
                print(f"Error loading {file}: {e}")
    
    if all_data:
        df_all = pd.concat(all_data).reset_index(drop=True)
        print("Combined Data Shape:", df_all.shape)
        print(df_all[['ticker', 'timestamp', 'open', 'high', 'low', 'close']].head())
        return df_all
    else:
        print("No data loaded!")
        return None

def get_atm_strike_and_prices(df_day):
    """
    Identify the ATM strike and return its CE/PE tickers
    """
    window_df = df_day[df_day['timestamp'].dt.strftime('%H:%M:%S').between("09:30:00", "09:31:59")]
    window_df = window_df[window_df['ticker'].str.startswith("NIFTY")].copy()

    if window_df.empty:
        return None, None, None

    window_df['strike'] = window_df['ticker'].str.extract(r'NIFTY\d{2}[A-Z]{3}\d{2}(\d{4,5})').astype(float)
    window_df['option_type'] = window_df['ticker'].str.extract(r'(CE|PE)')

    pivot = window_df.pivot_table(index='strike', columns='option_type', values='close', aggfunc='mean')
    pivot = pivot.dropna()
    pivot['diff'] = abs(pivot['CE'] - pivot['PE'])

    # Print the pivot table for reference
    print("\nPivot Table (CE vs PE premiums for each strike):")
    print(pivot)

    atm_strike = pivot['diff'].idxmin()

    ce_ticker = window_df[(window_df['strike'] == atm_strike) & (window_df['option_type'] == 'CE')]['ticker'].iloc[0]
    pe_ticker = window_df[(window_df['strike'] == atm_strike) & (window_df['option_type'] == 'PE')]['ticker'].iloc[0]

    print(f"ATM Strike: {atm_strike}")
    print(f"CE Ticker: {ce_ticker}")
    print(f"PE Ticker: {pe_ticker}")

    return atm_strike, ce_ticker, pe_ticker

def backtest_short_straddle(df_all, date):
    """
    Backtest short straddle strategy for a specific date
    """
    print(f"\n=== Backtesting for {date} ===")
    df_day = df_all[df_all['timestamp'].dt.date == pd.to_datetime(date).date()]
    df_day = df_day[df_day['ticker'].str.startswith("NIFTY")]

    if df_day.empty:
        print(f"No data available for {date}")
        return None

    atm_strike, ce_ticker, pe_ticker = get_atm_strike_and_prices(df_day)
    if not atm_strike:
        print(f"Could not determine ATM strike for {date}")
        return None

    ce_df = df_day[df_day['ticker'] == ce_ticker]
    pe_df = df_day[df_day['ticker'] == pe_ticker]

    ce_entry_row = ce_df[ce_df['timestamp'].dt.strftime('%H:%M:%S') == ENTRY_TIME]
    pe_entry_row = pe_df[pe_df['timestamp'].dt.strftime('%H:%M:%S') == ENTRY_TIME]

    if ce_entry_row.empty or pe_entry_row.empty:
        print(f"Entry time data not available at {ENTRY_TIME} for {date}")
        return None

    # Short straddle: SELL CE and PE options at entry
    ce_entry = ce_entry_row['close'].iloc[0]
    pe_entry = pe_entry_row['close'].iloc[0]
    gross_entry = ce_entry + pe_entry  # Premium received from selling both options
    
    # For short straddle (sell options), we lose money when premium increases
    # and make money when premium decreases
    sl_premium = gross_entry * (1 + SL_PCT)  # SL triggered if combined premium increases by SL_PCT%
    
    print(f"CE Entry (SELL): {ce_entry}")
    print(f"PE Entry (SELL): {pe_entry}")
    print(f"Gross Premium Received: {gross_entry}")
    print(f"Stop Loss Premium Level: {sl_premium} (Risk: {SL_PCT*100}%)")

    merged = pd.merge(
        ce_df[['timestamp', 'close']],
        pe_df[['timestamp', 'close']],
        on='timestamp',
        suffixes=('_ce', '_pe')
    ).sort_values('timestamp')

    trades = []
    active_trade = True

    for _, row in merged.iterrows():
        ts = row['timestamp']
        if ts.strftime('%H:%M:%S') < ENTRY_TIME:
            continue
        if ts.strftime('%H:%M:%S') > EXIT_TIME:
            break

        curr_premium = row['close_ce'] + row['close_pe']
        
        # Check for stop loss hit - for short positions, SL hits when premium increases
        if active_trade and curr_premium >= sl_premium:
            # Calculate PnL for a short position: (entry - exit) * lotsize - brokerage
            # For short straddle, we SOLD options at entry and are BUYING them back at exit
            pnl = (gross_entry - curr_premium - (2 * BROKERAGE_PER_LEG)) * LOTSIZE
            trades.append({
                'date': date,
                'entry_time': ENTRY_TIME,
                'exit_time': ts,
                'ce_entry': ce_entry,
                'pe_entry': pe_entry,
                'ce_exit': row['close_ce'],
                'pe_exit': row['close_pe'],
                'exit_reason': 'SL_HIT',
                'PnL': pnl
            })
            active_trade = False
            print(f"SL hit at {ts.strftime('%H:%M:%S')} - PnL: {pnl}")
            break

    # If we made it to market close without hitting SL
    if active_trade:
        last = merged[merged['timestamp'].dt.strftime('%H:%M:%S') <= EXIT_TIME].iloc[-1]
        curr_premium = last['close_ce'] + last['close_pe']
        # Calculate PnL for a short position: (entry - exit) * lotsize - brokerage
        pnl = (gross_entry - curr_premium - (2 * BROKERAGE_PER_LEG)) * LOTSIZE
        trades.append({
            'date': date,
            'entry_time': ENTRY_TIME,
            'exit_time': last['timestamp'],
            'ce_entry': ce_entry,
            'pe_entry': pe_entry,
            'ce_exit': last['close_ce'],
            'pe_exit': last['close_pe'],
            'exit_reason': 'MARKET_CLOSE',
            'PnL': pnl
        })
        print(f"Position closed at market close - PnL: {pnl}")

    return pd.DataFrame(trades)

def generate_performance_report(results_df):
    """
    Generate performance metrics from backtest results
    """
    results_df['date'] = pd.to_datetime(results_df['date'])
    results_df = results_df.sort_values('date')

    results_df['cum_pnl'] = results_df['PnL'].cumsum()
    results_df['equity'] = INITIAL_CAPITAL + results_df['cum_pnl']
    results_df['daily_return'] = results_df['PnL'] / results_df['equity'].shift(1)
    results_df['daily_return'] = results_df['daily_return'].fillna(0)

    # === Performance Metrics ===
    total_days = len(results_df)
    profitable_days = sum(results_df['PnL'] > 0)
    win_rate = profitable_days / total_days if total_days > 0 else 0
    
    # Segregate by exit reason
    sl_hits = sum(results_df['exit_reason'] == 'SL_HIT')
    market_closes = sum(results_df['exit_reason'] == 'MARKET_CLOSE')
    
    total_return = results_df['equity'].iloc[-1] - INITIAL_CAPITAL
    percentage_return = (total_return / INITIAL_CAPITAL) * 100
    
    max_drawdown = (results_df['equity'].cummax() - results_df['equity']).max()
    max_drawdown_pct = max_drawdown / results_df['equity'].cummax().max() * 100
    
    # Annualized metrics (assuming 252 trading days in a year)
    days_elapsed = (results_df['date'].iloc[-1] - results_df['date'].iloc[0]).days
    years = days_elapsed / 365
    cagr = ((results_df['equity'].iloc[-1] / INITIAL_CAPITAL) ** (1 / years if years > 0 else 1)) - 1
    sharpe = results_df['daily_return'].mean() / results_df['daily_return'].std() * np.sqrt(252) if results_df['daily_return'].std() > 0 else 0

    print("\n==== Performance Summary ====")
    print(f"Total Days     : {total_days}")
    print(f"Win Rate       : {win_rate:.2%}")
    print(f"Total Return   : ₹{total_return:.2f} ({percentage_return:.2f}%)")
    print(f"Max Drawdown   : ₹{max_drawdown:.2f} ({max_drawdown_pct:.2f}%)")
    print(f"CAGR           : {cagr * 100:.2f}%")
    print(f"Sharpe Ratio   : {sharpe:.2f}")
    
    print("\n==== Exit Statistics ====")
    print(f"Stop Loss Hits : {sl_hits} ({sl_hits/total_days:.2%} of trades)")
    print(f"Market Close   : {market_closes} ({market_closes/total_days:.2%} of trades)")

    # === Daily Table ===
    summary_table = results_df[['date', 'exit_reason', 'PnL', 'equity']].copy()
    summary_table.columns = ['Date', 'Exit Reason', 'Daily PnL', 'Capital']
    summary_table['Date'] = summary_table['Date'].dt.strftime('%Y-%m-%d')
    summary_table['Daily PnL'] = summary_table['Daily PnL'].map(lambda x: f"{x:+,.2f}")
    summary_table['Capital'] = summary_table['Capital'].map(lambda x: f"{x:,.2f}")

    print("\n==== Daily Capital & PnL ====")
    print(summary_table.to_string(index=False))

    return results_df

def run_backtest(start_date, end_date):
    """
    Run the backtest for a specified date range
    """
    print(f"==== Running Short Straddle Backtest ====")
    print(f"Date Range: {start_date} to {end_date}")
    print(f"Entry Time: {ENTRY_TIME}")
    print(f"Exit Time: {EXIT_TIME}")
    print(f"Lot Size: {LOTSIZE}")
    print(f"Stop Loss: {SL_PCT*100}% of entry premium")
    print(f"Initial Capital: ₹{INITIAL_CAPITAL:,}")
    
    # Load data
    df_all = load_data(DATA_FOLDER)
    if df_all is None:
        return
    
    # Get all trading days in the specified range
    all_dates = pd.date_range(start=start_date, end=end_date, freq='B')
    all_trades = []

    for d in all_dates:
        day_result = backtest_short_straddle(df_all, d.strftime('%Y-%m-%d'))
        if day_result is not None:
            all_trades.append(day_result)

    if all_trades:
        results = pd.concat(all_trades)
        final_df = generate_performance_report(results)
        
        # Export results to CSV
        results_filename = f"ShortStraddle_Results_{start_date}to{end_date}.csv"
        results.to_csv(results_filename, index=False)
        print(f"\nResults exported to {results_filename}")
        
        return final_df
    else:
        print("No trades executed during the backtest period.")
        return None

# === MAIN EXECUTION ===
if __name__ == "__main__":
    start_date = '2021-04-01'
    end_date = '2021-04-30'
    
    results_df = run_backtest(start_date, end_date)
    
    if results_df is not None:
        # You can add code here to create plots or additional analysis
        print("\nBacktest completed successfully!")
    else:
        print("\nBacktest failed to execute properly.")

==== Running Short Straddle Backtest ====
Date Range: 2021-04-01 to 2021-04-30
Entry Time: 09:30:59
Exit Time: 15:15:00
Lot Size: 75
Stop Loss: 10.0% of entry premium
Initial Capital: ₹1,000,000
Combined Data Shape: (12459881, 10)
                      ticker           timestamp     open     high      low  \
0       ACC29APR211900CE.NFO 2021-04-01 09:15:59    78.05    79.00    78.00   
1       ACC29APR211940CE.NFO 2021-04-01 09:15:59    59.95    60.00    59.95   
2                  ACC-I.NFO 2021-04-01 09:15:59  1923.00  1929.55  1920.00   
3  ADANIENT29APR211020CE.NFO 2021-04-01 09:15:59    81.85    81.85    81.85   
4  ADANIENT29APR211020PE.NFO 2021-04-01 09:15:59    52.90    52.90    52.90   

     close  
0    79.00  
1    60.00  
2  1923.95  
3    81.85  
4    52.90  

=== Backtesting for 2021-04-01 ===

Pivot Table (CE vs PE premiums for each strike):
option_type           CE           PE         diff
strike                                            
12150.0      2690.000000    